In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import math
from core.objects.Timestamp import Timestamp as Timestamp

data = pd.read_csv("data/timings/District_WestHam-TurnhamGreen.csv")

data = data.reset_index()  # make sure indexes pair with number of rows

Date = "20/01/2022"
TimeStep = 0.5 #s
TubeLine = 'D'
InsertDataToDB = False #Only set to true once file has been run, the TimeStamps DataFrame looks correct and no errors are thrown

Stations = pd.DataFrame()
Stations['PrevStation'] = data['Segment'].shift(1)
Stations['Departure'] = data['Departure'].shift(1)
Stations['NextStation'] = data['Segment']
Stations['Arrival'] = data['Arrival']

StationNames = []
TimeStamps = pd.DataFrame(columns=['index', 'Timestamp', 'FromStation', 'ToStation', 'TubeLine'])
for index, row in Stations.iterrows():
    FromStation = row['PrevStation']
    ToStation = row['NextStation']

    if (isinstance(FromStation, str) and FromStation not in StationNames):
        StationNames.append(FromStation)

    if (isinstance(ToStation, str) and ToStation not in StationNames):
        StationNames.append(ToStation)

    DeptTime = row['Departure']
    if (not isinstance(DeptTime, str)):
        continue
    DeptDateTimeString = Date + ' ' +DeptTime
    DepartureTimestamp = time.mktime(datetime.datetime.strptime(DeptDateTimeString, "%d/%m/%Y %H:%M:%S").timetuple())


    ArrTime = row['Arrival']
    if (not isinstance(ArrTime, str)):
        continue
    ArrDateTimeString = Date + ' ' +ArrTime
    ArrivalTimestamp = time.mktime(datetime.datetime.strptime(ArrDateTimeString, "%d/%m/%Y %H:%M:%S").timetuple())

    if (index < len(Stations.index)-1):
        NextDeptTime = Stations['Departure'][index + 1]
        if (not isinstance(NextDeptTime, str)):
            continue
        NextDeptDateTimeString = Date + ' ' +NextDeptTime
        NextDeptTimeStamp = time.mktime(datetime.datetime.strptime(NextDeptDateTimeString, "%d/%m/%Y %H:%M:%S").timetuple())
    else:
        NextDeptTimeStamp = ArrivalTimestamp + 60

    Time = DepartureTimestamp
    Index = 0
    while Time <= ArrivalTimestamp:
        TimeStamps = pd.concat([TimeStamps, pd.DataFrame.from_records([{
            'index': Index,
            'Timestamp': Time,
            'FromStation': FromStation,
            'ToStation': ToStation,
            'TubeLine': TubeLine,
        }])])

        TS = Timestamp(Time, TubeLine, FromStation, ToStation, Index)
        TS.checkStationNames()
        
        if InsertDataToDB:
            TS.insert()

        Time += TimeStep
        Index += 1


    Index = 0
    while Time < NextDeptTimeStamp:
        TimeStamps = pd.concat([TimeStamps, pd.DataFrame.from_records([{
            'index': Index,
            'Timestamp': Time,
            'FromStation': ToStation,
            'ToStation': ToStation,
            'TubeLine': TubeLine,
        }])])

        TS = Timestamp(Time, TubeLine, FromStation, ToStation, Index)
        TS.checkStationNames()
        
        if InsertDataToDB:
            TS.insert()

        Time += TimeStep
        Index += 1
        
print(StationNames)

['West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green', 'Whitechapel', 'Aldgate East', 'Tower Hill', 'Monument', 'Cannon Street', 'Mansion House', 'Blackfriars', 'Temple', 'Embankment', 'Westminster', "St James's Park", 'Victoria', 'Sloane Square', 'South Kensington', 'Gloucester Road', "Earl's Court", 'West Kensington', 'Barons Court', 'Hammersmith', 'Ravenscourt Park', 'Stamford Brook', 'Turnham Green']
